## Pymssql 테스트 
* _mssql 예제 : http://pymssql.org/en/latest/_mssql_examples.html
* pymssql 예제 : http://pymssql.org/en/latest/pymssql_examples.html

In [1]:
import _mssql

# 아래의 with 문을 사용하게 되면 conn.close() 를 별도 호출할 필요가 없음
# 일반적인 코드 : conn = _mssql.connect(server='172.16.220.240', user='usrPython', password='vkdlTjs', database='DSDBA')

with _mssql.connect(server='172.16.220.240', user='usrPython', password='vkdlTjs', database='DSDBA') as conn:
    conn.execute_non_query("if object_id('persons') is not null DROP TABLE persons")

    conn.execute_non_query('CREATE TABLE persons(id INT, name VARCHAR(100))')
    conn.execute_non_query("INSERT INTO persons VALUES(1, 'John Doe')")
    conn.execute_non_query("INSERT INTO persons VALUES(2, 'Jane Doe')")

    conn.execute_non_query("DROP TABLE persons")

print("성공!!!")

성공!!!


In [12]:
with _mssql.connect(server='172.16.220.240', user='usrPython', password='vkdlTjs', database='DSDBA') as conn:
    conn.execute_query("SELECT code, codename FROM code WHERE useYN='Y' and code < 11")
    for row in conn:
        print('code : {0}, codename : {1}'.format(row['code'], row['codename']))

code : 1, codename : NONE
code : 2, codename : MSCS
code : 3, codename : RAC
code : 4, codename : Auto Start
code : 5, codename : N/A
code : 6, codename : 1등급
code : 7, codename : 2등급
code : 8, codename : 3등급
code : 9, codename : 4등급
code : 10, codename : 5등급


In [15]:
with _mssql.connect(server='172.16.220.240', user='usrPython', password='vkdlTjs', database='DSDBA') as conn:

    # examples of other query functions
    numemployees = conn.execute_scalar("SELECT COUNT(*) FROM code")
    numemployees = conn.execute_scalar("SELECT COUNT(*) FROM code WHERE code in (1, 2, 3, 5)")
    employeedata = conn.execute_row("SELECT code, codename FROM code WHERE code > 1")

print(conn.connected)

False


In [5]:
with _mssql.connect(server='172.16.220.240', user='usrPython', password='vkdlTjs', database='DSDBA') as conn:

    conn.execute_query('sp_spaceused')   # sp_spaceused without arguments returns 2 result sets
    res1 = [ row for row in conn ]       # 1st result
    res2 = [ row for row in conn ]       # 2nd result

    results = res1 + res2

    results

In [16]:
# 에러처리~
with _mssql.connect(server='172.16.220.240', user='usrPython', password='vkdlTjs', database='DSDBA') as conn:
    conn.execute_non_query("if object_id('t1') is not null drop table t1; CREATE TABLE t1(id INT, name VARCHAR(50))")

    try:
        conn.execute_non_query('CREATE TABLE t1(id INT, name VARCHAR(50))')
    except _mssql.MssqlDatabaseException as e:
        if e.number == 2714 and e.severity == 16:
            # table already existed, so quieten the error
            print("Table already existed!!!")
        else:
            raise # re-raise real error
            print("Drop and recreate!!")
    finally:
        conn.close()

Table already existed!!!


## 이건 실행 안되는 예제 (수정해야 함)

In [ ]:
from os import getenv
import pymssql

server = getenv("PYMSSQL_TEST_SERVER")
user = getenv("PYMSSQL_TEST_USERNAME")
password = getenv("PYMSSQL_TEST_PASSWORD")

conn = pymssql.connect(server, user, password, "tempdb")
cursor = conn.cursor()
cursor.execute("""
IF OBJECT_ID('persons', 'U') IS NOT NULL
    DROP TABLE persons
CREATE TABLE persons (
    id INT NOT NULL,
    name VARCHAR(100),
    salesrep VARCHAR(100),
    PRIMARY KEY(id)
)
""")
cursor.executemany(
    "INSERT INTO persons VALUES (%d, %s, %s)",
    [(1, 'John Smith', 'John Doe'),
     (2, 'Jane Doe', 'Joe Dog'),
     (3, 'Mike T.', 'Sarah H.')])
# you must call commit() to persist your data if you don't set autocommit to True
conn.commit()

cursor.execute('SELECT * FROM persons WHERE salesrep=%s', 'John Doe')
row = cursor.fetchone()
while row:
    print("ID=%d, Name=%s" % (row[0], row[1]))
    row = cursor.fetchone()

conn.close()

### Connecting using Windows Authentication

In [ ]:
conn = pymssql.connect(
    host=r'dbhostname\myinstance',
    user=r'companydomain\username',
    password=PASSWORD,
    database='DatabaseOfInterest'
)

### Iterating through results

In [ ]:
conn = pymssql.connect(server, user, password, "tempdb")
cursor = conn.cursor()
cursor.execute('SELECT * FROM persons WHERE salesrep=%s', 'John Doe')

for row in cursor:
    print('row = %r' % (row,))

conn.close()

### Important note about Cursors

In [ ]:
c1 = conn.cursor()
c1.execute('SELECT * FROM persons')

c2 = conn.cursor()
c2.execute('SELECT * FROM persons WHERE salesrep=%s', 'John Doe')

print( "all persons" )
print( c1.fetchall() )  # shows result from c2 query!

print( "John Doe" )
print( c2.fetchall() )  # shows no results at all!

In [ ]:
c1.execute('SELECT ...')
c1_list = c1.fetchall()

c2.execute('SELECT ...')
c2_list = c2.fetchall()

# use c1_list and c2_list here instead of fetching individually from
# c1 and c2

### Rows as dictionaries

In [ ]:
conn = pymssql.connect(server, user, password, "tempdb")
cursor = conn.cursor(as_dict=True)

cursor.execute('SELECT * FROM persons WHERE salesrep=%s', 'John Doe')
for row in cursor:
    print("ID=%d, Name=%s" % (row['id'], row['name']))

conn.close()

### Using With

In [ ]:
with pymssql.connect(server, user, password, "tempdb") as conn:
    with conn.cursor(as_dict=True) as cursor:
        cursor.execute('SELECT * FROM persons WHERE salesrep=%s', 'John Doe')
        for row in cursor:
            print("ID=%d, Name=%s" % (row['id'], row['name']))

### Stored Procedure 사용

In [ ]:
with pymssql.connect(server, user, password, "tempdb") as conn:
    with conn.cursor(as_dict=True) as cursor:
        cursor.execute("""
        CREATE PROCEDURE FindPerson
            @name VARCHAR(100)
        AS BEGIN
            SELECT * FROM persons WHERE name = @name
        END
        """)
        cursor.callproc('FindPerson', ('Jane Doe',))
        for row in cursor:
            print("ID=%d, Name=%s" % (row['id'], row['name']))